# 2/

Entity resolution: 
- deal with the situation that:
  -  there are duplicates in entities because of the label-name mismacth, they are included because their pages can be accessed via multiple labels
- owl:sameAs

deal with data

don't put it here, because we need to deal with merged_influence previously

In [ ]:
# correct the labels and displayed labels
# it should be okay because it is recorded in dbpedia_info label
ent_info_list = pickle.load( open("info_lists/db_philo_info_infludict_list.p", "rb") )

ent_correct_dict = {} # key: entity (wrong label), value: correct label
for item in ent_info_list:
    if item["dbpedia_info"]['wikiPageID']:
        # print(item["entity"], item["wiki_url"],item["dbpedia_info"]["wikiPageID"])
        page = requests.get(item["wiki_url"])
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="firstHeading")
        entity_corrected = results.text
        
        if item["entity"] != entity_corrected:
            print(item["entity"], "=>",entity_corrected)
        ent_correct_dict[item["entity"]] = entity_corrected

# or, if we don't want a check, just delete all of them
# print(ent_correct_dict)
with open('check_lists/ent_correct_dict.json', 'w') as outfile:
    json.dump(ent_correct_dict, outfile, indent=4)

In [ ]:
# haven't consider duplicates
# haven't consider entity resolution 

In [ ]:
ent_info_list = pickle.load( open("info_lists/db_philo_info_infludict_list.p", "rb") )

for item in ent_info_list:
    item["accessible_label"] = []
    
    ent = item["entity"]
    label = item["dbpedia_info"]["label"] # string
    if label != ent:
        item["entity"] = label
        print(item["entity"], "<-",ent)
        item["accessible_label"].append(label)

    item["accessible_label"].append(ent)
    print(item["accessible_label"])

with open('check_lists/ent_entToLabel_info_list.json', 'w') as outfile:
    json.dump(ent_info_list, outfile, indent=4)

['Lopamudra']
['Maitreyi']
['Ghosha']
['Gargi Vachaknavi']
Theano (philosopher) <- Theano of Croton
['Theano (philosopher)', 'Theano of Croton']
Themistoclea <- Aristoclea of Delphi
['Themistoclea', 'Aristoclea of Delphi']
Khujjuttara <- Khujjuttarā
['Khujjuttara', 'Khujjuttarā']
Aspasia <- Aspasia of Miletus
['Aspasia', 'Aspasia of Miletus']
['Arete of Cyrene']
['Hipparchia of Maroneia']
['Nicarete of Megara']
['Catherine of Alexandria']
['Ptolemais of Cyrene']
Aesara <- Aesara of Lucania
['Aesara', 'Aesara of Lucania']
['Diotima of Mantinea']
['Ban Zhao']
Sosipatra <- Sosipatra of Ephesus
['Sosipatra', 'Sosipatra of Ephesus']
['Xie Daoyun']
['Hypatia']
['Aedesia']
['Theodora']
Maṇḍana Miśra <- Ubhaya Bharati
['Maṇḍana Miśra', 'Ubhaya Bharati']
Héloïse <- Héloïse d'Argenteuil
['Héloïse', "Héloïse d'Argenteuil"]
['Akka Mahadevi']
['Marguerite Porete']
["Tullia d'Aragona"]
['Lalleshwari']
['Catherine of Siena']
['Moderata Fonte']
['Hildegard of Bingen']
['Teresa of Ávila']
['Mary Astell

more entity resolution

In [ ]:
# dbo:wikiPageDisambiguates
disambig_dict = {}
prob_redirect = []
for ent in list(ent_correct_dict.values()):
    db_url = "https://dbpedia.org/page/"+ent.replace(" ","_")
    # page = requests.get(db_url, allow_redirects=False)
    try:
        page = requests.get(db_url)
    except:
        prob_redirect.append(db_url)
        continue
    info_df = pd.read_html(page.content)[0].values

    for i in info_df:
        text = re.sub('\s+', '', i[1])
        if i[0] == "dbo:wikiPageDisambiguates":
            print(ent, text.split("dbr:")[1:])
            disambig_dict[ent] = text.split("dbr:")[1:]

with open('check_lists/disambig_dict.json', 'w') as outfile:
    json.dump(disambig_dict, outfile, indent=4)

print(len(disambig_dict))
prob_redirect

Theodora ['Theodora_(Handel)', 'Theodora_(empress)', 'Theodora_(wife_of_Justinian_I)', 'Theodora_(given_name)', 'Theodora_Kantakouzene', 'Alexios_I_of_Trebizond', 'Princess_Theodora_of_Greece_and_Denmark', 'Theodora_and_Didymus', 'Princess_Theodora_of_Greece_and_Denmark_(1906–1969)', 'Theodora_(daughter_of_Constantine_VII)', 'Theodora_(wife_of_Romanos_I)', 'Theodora_of_Arta', 'Theodora_(1921_film)', 'Theodora_Porphyrogenita', 'Theodora_(senatrix)', 'Theodora_of_Khazaria', 'Theodora_(Roman_martyr)', 'Theodora_of_Trebizond', 'Flavia_Maximiana_Theodora', "Theodora_Children's_Charity", 'Theodora_(1914_film)', 'Theodora_Komnene', 'Theodora_Kantakouzene_(wife_of_Alexios_IV_of_Trebizond)', 'Theodora_Tocco', 'Theodora_Palaiologina_(Byzantine_empress)', 'Theodora_of_Alexandria', 'Episcopa_Theodora', 'Theodora_of_Emesa', 'Theodora_of_Sihla']
Anne Conway ['Anne_Conway_(philosopher)']
Elisabeth of Bohemia ['Elizabeth_of_Bohemia_(1292–1330)', 'Elizabeth_of_Luxembourg', 'Elisabeth_of_Bohemia_(1358–1

['https://dbpedia.org/page/Jeong_Yak-yong',
 'https://dbpedia.org/page/Diogenes_Laertius',
 'https://dbpedia.org/page/Eugen_Fink']

clean duplicates derived from multiple wrong referred names

In [ ]:
# they are included because their pages can be accessed via multiple labels
# we access each link and get the label directly from the page
with open('check_lists/ent_entToLabel_info_list.json') as json_file:
    ent_info_list = json.load(json_file)

# print the wrong entities
l = [item["dbpedia_info"]['wikiPageID'] for item in ent_info_list]

visited = set()
dup = [x for x in l if x in visited or (visited.add(x) or False)]
# print(dup)
dup = [i for i in dup if i]
print(dup)

ent_dupl_dict = {} # key: entity (wrong label), value: correct label
for item in ent_info_list:
    if item["dbpedia_info"]['wikiPageID'] in dup:
        print(item["entity"], item["accessible_label"])
        ent_dupl_dict[item["entity"]] = 0

# or, if we don't want a check, just delete all of them
print(ent_dupl_dict)

COUNT = 0
for item in ent_info_list:
    # if it is duplicated
    if item["dbpedia_info"]['wikiPageID'] in dup and ent_dupl_dict[item["entity"]] == 0:
        ent_dupl_dict[item["entity"]] = 1
        ent_info_list.remove(item)
        # print(ent_dupl_dict[item["entity"]])
        COUNT += 1
        continue
# check
print(ent_dupl_dict)
if len(ent_dupl_dict) == COUNT:
    print("Duplicates cleaned")
    path = 'check_lists/ent_remDupl_info_list.json'
    with open(path, 'w') as outfile:
        json.dump(ent_info_list, outfile, indent=4)
    print("Updated data saved: " + path)

['1034793', '2157753', '38375', '2245936', '14125745', '177557', '3783415', '43583', '2016044']
Hypatia ['Hypatia']
Teresa of Ávila ['Teresa of Ávila']
Catharine Trotter Cockburn ['Catharine Trotter Cockburn']
Susan Stebbing ['Susan Stebbing']
Mary Warnock, Baroness Warnock ['Mary Warnock, Baroness Warnock']
Isaac Abarbanel ['Isaac Abarbanel', 'Isaac ben Judah Abravanel']
Isaac Abarbanel ['Isaac Abarbanel']
Catharine Trotter Cockburn ['Catharine Trotter Cockburn', 'Catherine Trotter Cockburn']
Guo Xiang ['Guo Xiang']
Hypatia ['Hypatia', 'Hypatia of Alexandria']
Friedrich Wilhelm Joseph Schelling ['Friedrich Wilhelm Joseph Schelling']
Guo Xiang ['Guo Xiang', 'Kuo Hsiang']
Philo the Dialectician ['Philo the Dialectician']
Philo the Dialectician ['Philo the Dialectician', 'Philo of Megara']
Friedrich Wilhelm Joseph Schelling ['Friedrich Wilhelm Joseph Schelling', 'Friedrich Schelling']
Susan Stebbing ['Susan Stebbing', 'L. Susan Stebbing']
Teresa of Ávila ['Teresa of Ávila', 'Teresa of Av

In [ ]:
for item in ent_info_list:
    item["accessible_label"] = [item["entity"]]
    item["refered_name"] = [item["refered_name"]]
    new_dict = {}

    # if item["entity"] in ent_correct_dict: # if included in the problematic lists
    # if it is duplicated
    if item["dbpedia_info"]['wikiPageID'] in dup:
        
        if item["entity"] in ent_correct_inv_dict: # delete the nodes 
            ent_info_list.remove(item)
            continue
        else: # correct those whose labels are wrong
            ent = item["entity"]
            item["entity"] = ent_correct_dict[ent] # correct it
            print(ent, "=>",item["entity"])
            item["wiki_url"] = "https://en.wikipedia.org/wiki/"+item["entity"].replace(" ","_")
            item["db_url"] = "https://dbpedia.org/page/"+item["entity"].replace(" ","_")

            item["accessible_label"] = ent_correct_inv_dict[ item["entity"] ]
            # it is also a check of the correctness of the corrected entities
            # item['dbpedia_info'] = get_db_philosopher( item['entity'])

Isaac ben Judah Abravanel => Isaac Abarbanel
Catherine Trotter Cockburn => Catharine Trotter Cockburn
Hypatia of Alexandria => Hypatia
Kuo Hsiang => Guo Xiang
Philo of Megara => Philo the Dialectician
Friedrich Schelling => Friedrich Wilhelm Joseph Schelling
L. Susan Stebbing => Susan Stebbing
Teresa of Avila => Teresa of Ávila
Mary Warnock => Mary Warnock, Baroness Warnock


modify influence lists

In [ ]:
# map entity and wikipageid
with open('check_lists/ent_remDupl_info_list.json') as json_file:
    ent_info_list = json.load(json_file)

ent_id_dict, id_ent_dict = {}, {}
for item in ent_info_list:
    if item['dbpedia_info']['wikiPageID'] :
        id = int( item['dbpedia_info']['wikiPageID'] )
    ent = item['entity']
    ent_id_dict[ent] = id
    id_ent_dict[id] = ent
# id_ent_dict

for item in ent_info_list:
    # reformat
    for key in item["dbpedia_info"]["db_influence_record"]: # key
        l = item["dbpedia_info"]["db_influence_record"][key]
        # print(l)
        item["dbpedia_info"]["db_influence_record"][key] = [i.replace("_"," ") for i in l]
    # print(item["dbpedia_info"]["db_influence_record"])

        # add id 
        info_list = []
        for ent in item["dbpedia_info"]["db_influence_record"][key] :
            if ent in ent_id_dict:
                info_dict = {"entity": ent, "wikiPageID": ent_id_dict[ent]}
            else:
                info_dict = {"entity": ent, "wikiPageID": ""}
            info_list.append(info_dict)
        # print(info_list)
        item["dbpedia_info"]["db_influence_record"][key] = info_list
        

In [ ]:
with open('info_lists/philo_info_list.json', 'w') as outfile:
    json.dump(ent_info_list, outfile, indent=4)

Deal with birthYear


In [ ]:
with open('info_lists/philo_info_list.json', 'r') as f:
  ent_info_list = json.load(f)

# db records
for item in ent_info_list:
    if item["dbpedia_info"]["birthYear"]:
        # print(item["dbpedia_info"]["birthYear"][:4])
        item["dbpedia_info"]["birthYear"] = item["dbpedia_info"]["birthYear"][:4]
        print(item["dbpedia_info"]["birthYear"])

-350
0287
0049
1100
0013
1501
1320
1347
1098
1659
1802
1831
1822
1825
1623
1943
1976
1861
1946
1919
1906
1929
1955
1965
1935
1908
1950
1978
1956
1863
1946
1941
1940
1957
1947
1943
1937
1933
1967
1970
1910
1941
1920
1947
1941
1978
1939
1945
1935
1946
1929
1929
1924
1951
1954
1943
1930
1942
1934
1952
1941
1895
1961
1944
1950
1950
1921
1919
1933
1919
1955
1929
1947
1867
1958
1948
1944
1959
1879
1932
1962
1952
1953
1942
1891
1949
1947
1968
1929
1948
1909
1952
1954
1904
1966
1955
1901
1079
1939
1437
1860
1672
1080
1903
0001
0001
1486
1921
1200
1859
1935
1921
1918
0175
1703
0500
0610
-586
-446
1922
1225
0316
0435
-435
-384
1928
1926
1920
0354
1911
1843
1126
0980
1910
1884
0020
1937
1162
1917
1741
1944
1895
1814
1686
1942
1915
1313
1724
1897
1929
1809
1714
1647
1738
1761
1542
1930
1949
1945
1798
1892
1886
1748
1874
1906
1859
1909
1090
1909
1944
1718
1907
1892
1885
1942
1861
1909
1530
0477
1938
1575
1781
1754
1221
1930
1845
1847
1846
1900
1910
1838
1884
1887
1950
1889
1869
1878
1914
1824
1821


In [ ]:
with open('info_lists/philo_info_list.json', 'w') as outfile:
    json.dump(ent_info_list, outfile, indent=4)

## 3/ validate the invalidation in wikipedia


1. gender

Crawl women philosophers

Note: some links should not be included! For example, Héloïse d'Argenteuil (1090–1164), contributed to the ethical thought of Peter Abelard.

2. Philosopher? E.g., Megan Craig

    - check if there is philo in abstract?

In [ ]:
women_pronouns = [" she ", " She ", " her ", " Her"]
men_pronouns = [" he ", " He ", " his ", " His"]
pronouns = women_pronouns + men_pronouns

potential_list = []

for item in checked_philo_dbpedia_info_list:
        abstract = item['abstract']
        name = item['name']
        # let's assume that the sexual pronouns appear in the first three sentences
        # corner cases: sometimes the abstract is less than 3 sentences and reveal nothing about gender
        bio_list = nltk.tokenize.sent_tokenize(abstract)
        # if len(bio_list) <= 3:
        #         print(bio)
        bio = " | ".join(bio_list[0:3])
        print(bio)
        # if not any(x in bio for x in pronouns):
        #         print(bio)
        # if any(x in bio for x in women_pronouns):
        #         if any(x in bio for x in men_pronouns):
        #                 print(name)
        #                 print(bio)
        #         else:
        #                 potential_list.append(item)

Nicola Abbagnano (Italian: [niˈkɔla abbaɲˈɲano]; 15 July 1901 – 9 September 1990) was an Italian existential philosopher. | (en)
Muḥammad 'Abduh (1849 – 11 July 1905) (also spelled Mohammed Abduh, Arabic: محمد عبده‎) was an Egyptian Islamic scholar, jurist, theologian, Freemason, and writer. | Abduh was the author of Risālat al-Tawḥīd (Arabic: رسالة التوحيد‎; "The Theology of Unity") and a commentary on the Quran. | He briefly published, alongside Jamāl al-Dīn al-Afghānī, the Pan-Islamist anti-colonial journal Al-Urwah al-Wuthqa.
Peter Abelard (/ˈæbəlɑːrd/; French: Pierre Abélard; Latin: Petrus Abaelardus or Abailardus; c. 1079 – 21 April 1142) was a medieval French scholastic philosopher, leading logician, theologian, poet, composer and musician. | In philosophy he is celebrated for his logical solution to the problem of universals via nominalism and conceptualism and his pioneering of intent in ethics. | Often referred to as the “Descartes of the twelfth century”, he is considered a 

In [ ]:

women_philosopher_list = []
sus_list = []

for node in potential_list:
    res = node['description']
    if any(x in res for x in caution):
        # print(res)
        sus_node = node
        sus_list.append(sus_node)
    else:
        women_philosopher_list.append(node)

In [ ]:
print(len(sus_list), len(women_philosopher_list))

Thoughts

How to deal with the case that two philosophers influence each other (bidirectional)? 

In [ ]:
# For other aspects
non_philo_ent_list